In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.9.0")

import java.io.File
import java.io.BufferedInputStream
import java.net.HttpURLConnection
import java.net.URI
import java.nio.file.*
import java.util.zip.ZipInputStream
import kotlin.system.measureTimeMillis
import kotlinx.coroutines.*

In [2]:
// ── Logging ────────────────────────────────────────────────────

enum class LogLevel(val label: String, val color: String) {
    INFO ("INFO ",  "\u001B[36m"),   // cyan
    WARN ("WARN ",  "\u001B[33m"),   // yellow
    ERROR("ERROR",  "\u001B[31m"),   // red
    STEP ("STEP ",  "\u001B[35m"),   // magenta
    TIME ("TIME ",  "\u001B[32m"),   // green
}

val RESET = "\u001B[0m"
val BOLD  = "\u001B[1m"

/** Curried logger: pick a level, get back a (String) -> Unit printer. */
val log: (LogLevel) -> (String) -> Unit = { level ->
    { message ->
        val ts = java.time.LocalTime.now()
            .format(java.time.format.DateTimeFormatter.ofPattern("HH:mm:ss.SSS"))
        println("${level.color}[${level.label}]$RESET ${BOLD}$ts$RESET  $message")
    }
}

// Convenience aliases produced by currying
val info  = log(LogLevel.INFO)
val warn  = log(LogLevel.WARN)
val error = log(LogLevel.ERROR)
val step  = log(LogLevel.STEP)
val time  = log(LogLevel.TIME)

// ── Directory aliases ──────────────────────────────────────────

val scriptDir      = File(System.getProperty("user.dir"))
val doomRPG        = scriptDir.resolve("DoomRPG")
val game           = doomRPG.resolve("Game")
val temporary      = doomRPG.resolve("Temporary")
val brewBinaryDir  = temporary.resolve("Brew Binary")
val desktopBinDir  = temporary.resolve("Desktop Binary")
val filesDir       = temporary.resolve("Files")

// Will be assigned in Step 2
var brewBinary:    File = File("")
var desktopBinary: File = File("")

// Total-time accumulator
var totalElapsedMs = 0L

In [3]:
// ── Inline helper: run a step with logging + timing ───────────

inline fun runStep(
    number: Int,
    name:   String,
    block:  () -> Unit,
) {
    step("┌── Step $number: $name ─── START")
    val elapsed = measureTimeMillis {
        try {
            block()
        } catch (e: Exception) {
            error("Exception in step $number: ${e.message}")
            throw e
        }
    }
    totalElapsedMs += elapsed
    time("Step $number completed in ${elapsed}ms")
    step("└── Step $number: $name ─── END\n")
}

// ── Download a file from `url` into `targetDir`, returns saved File ─

fun downloadFile(url: String, targetDir: File): File {
    info("Downloading: $url")

    var connection = URI(url).toURL().openConnection() as HttpURLConnection
    connection.instanceFollowRedirects = true  // follow simple redirects
    connection.connect()

    // Handle 3xx redirects manually for cross-protocol (http→https) cases
    var redirects = 0
    while (connection.responseCode in 301..308 && redirects < 10) {
        val location = connection.getHeaderField("Location")
            ?: break
        warn("Redirect #${++redirects} → $location")
        connection.disconnect()
        connection = URI(location).toURL().openConnection() as HttpURLConnection
        connection.instanceFollowRedirects = true
        connection.connect()
    }

    require(connection.responseCode == 200) {
        "HTTP ${connection.responseCode} for $url"
    }

    // Derive filename from Content-Disposition or URL path
    val contentDisposition = connection.getHeaderField("Content-Disposition")
    val fileName = contentDisposition
        ?.substringAfter("filename=", "")
        ?.trim('"', ' ')
        ?.takeIf { it.isNotBlank() }
        ?: url.substringAfterLast('/').substringBefore('?')

    val outFile = targetDir.resolve(fileName)

    connection.inputStream.buffered().use { input ->
        outFile.outputStream().buffered().use { output ->
            input.copyTo(output)
        }
    }

    connection.disconnect()
    info("Saved → ${outFile.name} (${outFile.length() / 1024} KB)")
    return outFile
}

// ── Resolve latest GitHub release asset download URL (no hardcoded tag) ─

fun resolveLatestGitHubAssetUrl(owner: String, repo: String): String {
    // /releases/latest redirects to /releases/tag/<tag>
    // The API equivalent gives us asset URLs directly.
    val apiUrl = "https://api.github.com/repos/$owner/$repo/releases/latest"
    info("Querying GitHub API: $apiUrl")

    val connection = URI(apiUrl).toURL().openConnection() as HttpURLConnection
    connection.setRequestProperty("Accept", "application/vnd.github+json")
    connection.connect()

    require(connection.responseCode == 200) {
        "GitHub API returned HTTP ${connection.responseCode}"
    }

    val json = connection.inputStream.bufferedReader().readText()
    connection.disconnect()

    // Lightweight JSON parsing — extract first browser_download_url value
    // that points to a .zip file
    val pattern = Regex(""""browser_download_url"\s*:\s*"([^"]+\.zip)"""")
    val match = pattern.find(json)
        ?: throw IllegalStateException("No .zip asset found in latest release")

    val assetUrl = match.groupValues[1]
    info("Resolved latest asset → $assetUrl")
    return assetUrl
}

// ── Extract ZIP fully into a directory ──────────────────────────

fun extractZip(zipFile: File, destDir: File) {
    info("Extracting ${zipFile.name} → ${destDir.name}")
    ZipInputStream(BufferedInputStream(zipFile.inputStream())).use { zis ->
        generateSequence { zis.nextEntry }.forEach { entry ->
            val outFile = destDir.resolve(entry.name)
            if (entry.isDirectory) {
                outFile.mkdirs()
            } else {
                outFile.parentFile?.mkdirs()
                outFile.outputStream().buffered().use { out -> zis.copyTo(out) }
            }
            zis.closeEntry()
        }
    }
    info("Extraction complete (\u2713)")
}

// ── Extract ZIP but strip one leading folder level ──────────────

fun extractZipStripRoot(zipFile: File, destDir: File) {
    info("Extracting ${zipFile.name} → ${destDir.name} (stripping root folder)")
    ZipInputStream(BufferedInputStream(zipFile.inputStream())).use { zis ->
        generateSequence { zis.nextEntry }.forEach { entry ->
            // Remove the first path component  (e.g. "DoomRPG/file.txt" → "file.txt")
            val stripped = entry.name.substringAfter('/', "")
            if (stripped.isBlank()) {
                zis.closeEntry()
                return@forEach
            }
            val outFile = destDir.resolve(stripped)
            if (entry.isDirectory) {
                outFile.mkdirs()
            } else {
                outFile.parentFile?.mkdirs()
                outFile.outputStream().buffered().use { out -> zis.copyTo(out) }
            }
            zis.closeEntry()
        }
    }
    info("Extraction complete (\u2713)")
}

In [4]:
runStep(1, "Create folders") {

    val directories = listOf(doomRPG, game, temporary, brewBinaryDir, desktopBinDir, filesDir)

    directories.forEach { dir ->
        dir.mkdirs().also { created ->
            when {
                created       -> info("Created: ${dir.relativeTo(scriptDir)}")
                dir.exists()  -> warn("Already exists: ${dir.relativeTo(scriptDir)}")
                else          -> error("Failed to create: ${dir.relativeTo(scriptDir)}")
            }
        }
    }

    info("""
        |Folder structure created:
        |DoomRPG
        |  ├── Game
        |  └── Temporary
        |      ├── Brew Binary
        |      ├── Desktop Binary
        |      └── Files
    """.trimMargin())
}

[STEP ] 09:27:25.083  └── Step 1: Create folders ─── END



In [5]:
runStep(2, "Download necessary files") {

    runBlocking {
        val brewUrl   = "https://archive.org/download/doomrpg_brew/doomrpg.zip"
        val gitHubUrl = async(Dispatchers.IO) {
            resolveLatestGitHubAssetUrl("Erick194", "DoomRPG-RE")
        }

        // Launch both downloads concurrently
        val brewJob = async(Dispatchers.IO) {
            downloadFile(brewUrl, filesDir)
        }

        val desktopJob = async(Dispatchers.IO) {
            val url = gitHubUrl.await()
            downloadFile(url, filesDir)
        }

        brewBinary    = brewJob.await()
        desktopBinary = desktopJob.await()

        info("brewBinary    = ${brewBinary.absolutePath}")
        info("desktopBinary = ${desktopBinary.absolutePath}")
    }
}

[STEP ] 09:27:30.323  └── Step 2: Download necessary files ─── END



In [6]:
runStep(3, "Extract archives") {

    runBlocking {
        // Extract both archives concurrently
        val job1 = launch(Dispatchers.IO) {
            extractZip(brewBinary, brewBinaryDir)
        }

        val job2 = launch(Dispatchers.IO) {
            extractZipStripRoot(desktopBinary, desktopBinDir)
        }

        joinAll(job1, job2)
    }

    info("Brew Binary contents:    ${brewBinaryDir.listFiles()?.map { it.name }}")
    info("Desktop Binary contents: ${desktopBinDir.listFiles()?.map { it.name }}")
}

[STEP ] 09:27:30.739  └── Step 3: Extract archives ─── END



In [7]:
runStep(4, "Final preparations") {

    // Copy Desktop Binary contents → Game
    info("Copying Desktop Binary contents → Game")
    desktopBinDir.walkTopDown().filter { it.isFile }.forEach { src ->
        val rel  = src.relativeTo(desktopBinDir)
        val dest = game.resolve(rel)
        dest.parentFile?.mkdirs()
        src.copyTo(dest, overwrite = true)
        info("  ${rel}")
    }

    // Copy doomrpg.bar from Brew Binary → Game
    val barFile = brewBinaryDir.walkTopDown()
        .firstOrNull { it.name.equals("doomrpg.bar", ignoreCase = true) }

    if (barFile != null) {
        barFile.copyTo(game.resolve(barFile.name), overwrite = true)
        info("Copied ${barFile.name} → Game")
    } else {
        warn("doomrpg.bar not found in Brew Binary — searching all of Temporary")
        temporary.walkTopDown()
            .firstOrNull { it.name.equals("doomrpg.bar", ignoreCase = true) }
            ?.also {
                it.copyTo(game.resolve(it.name), overwrite = true)
                info("Copied ${it.name} → Game (found in ${it.parentFile.name})")
            }
            ?: error("doomrpg.bar not found anywhere under Temporary!")
    }

    // Run BarToZip.exe silently, pipe stdin so it doesn't block on "press any key"
    val barToZip = game.resolve("BarToZip.exe")
    require(barToZip.exists()) { "BarToZip.exe not found in Game folder" }

    info("Running BarToZip.exe silently…")
    val process = ProcessBuilder(barToZip.absolutePath)
        .directory(game)             // working directory = Game
        .redirectInput(ProcessBuilder.Redirect.PIPE)
        .redirectOutput(ProcessBuilder.Redirect.DISCARD)  // silent: discard stdout
        .redirectErrorStream(true)                         // merge stderr into stdout
        .start()

    // Immediately close stdin so the "Press any key" prompt is satisfied
    process.outputStream.close()

    val exitCode = process.waitFor()
    info("BarToZip.exe exited with code $exitCode")

    if (exitCode != 0) {
        warn("BarToZip.exe returned non-zero exit code: $exitCode")
    }

    // Verify the process is truly gone
    if (process.isAlive) {
        warn("BarToZip.exe still alive — force-destroying")
        process.destroyForcibly().waitFor()
    }

    info("Game folder contents: ${game.listFiles()?.map { it.name }}")
}

[STEP ] 09:27:32.122  └── Step 4: Final preparations ─── END



In [8]:
runStep(5, "Run the game") {

    // Open the DoomRPG folder in Explorer (will appear behind the game window)
    info("Opening DoomRPG folder in Explorer…")
    ProcessBuilder("explorer.exe", doomRPG.absolutePath)
        .directory(doomRPG)
        .start()

    // Small delay so Explorer opens first and ends up behind the game window
    Thread.sleep(1_000)

    // Launch DoomRPG.exe in foreground, attempt fullscreen via /max start
    val doomExe = game.resolve("DoomRPG.exe")
    require(doomExe.exists()) { "DoomRPG.exe not found in Game folder" }

    info("Launching DoomRPG.exe (maximized / foreground)…")
    ProcessBuilder(
        "cmd.exe", "/c",
        "start", "/max",           // /max  → launch maximized
        "DoomRPG",                 // window title  (required by start when path has quotes)
        "\"" + doomExe.absolutePath + "\""
    )
        .directory(game)
        .start()

    info("DoomRPG.exe launched — enjoy the game! 🎮")

    // ── Total time summary ──
    time("═══════════════════════════════════════")
    time("  Total time across all steps: ${totalElapsedMs}ms")
    time("                         ≈ ${"%.2f".format(totalElapsedMs / 1000.0)}s")
    time("═══════════════════════════════════════")
}

[STEP ] 09:27:33.409  └── Step 5: Run the game ─── END

